In [3]:
import os
import sys
import pathlib
import pickle
import math
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline 

import numpy as np
import pandas as pd

from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr_wls import ParCorrWLS 
from tigramite.independence_tests.regressionCI import RegressionCI
from tigramite import plotting as tp

from IPython.display import clear_output

In [4]:
data_path = "E:/UG4data/NHS_aggregated"
output_path = "D:/ProgramData/Jupyter Notebook/UG4/results/NHS/PCMCI+"

set([f for f in os.listdir(data_path)])

{'PRB001_aggregated.csv',
 'PRB003_aggregated.csv',
 'PRB102_aggregated.csv',
 'PRB103_aggregated.csv',
 'PRB104_aggregated.csv',
 'PRB105_aggregated.csv',
 'PRB201_aggregated.csv',
 'PRX018_aggregated.csv'}

In [5]:
df = pd.read_csv(f"{data_path}/PRB001_aggregated.csv")
df

,timestamp,breathingRate,activityLevel,activity_name,coughing,breathing,talking,hyperventilation,subject_id,long_id
0,2021-07-07 14:56:00,NaN,0.124682,Movement,0,0,0,0,PRB001,PRB001(1)
1,2021-07-07 14:57:00,16.119420,0.029450,Movement,0,75,0,0,PRB001,PRB001(1)
2,2021-07-07 14:58:00,20.930561,0.120812,Movement,0,0,0,0,PRB001,PRB001(1)
3,2021-07-07 14:59:00,18.133931,0.051179,Movement,0,0,25,0,PRB001,PRB001(1)
4,2021-07-07 15:00:00,22.970766,0.105806,Lying on right side,0,0,0,0,PRB001,PRB001(1)
...,...,...,...,...,...,...,...,...,...,...
23828,2021-07-17 19:57:00,24.206487,0.019234,Sitting/Standing,0,0,0,0,PRB001,PRB001(9)
23829,2021-07-17 19:58:00,23.736564,0.022812,Sitting/Standing,0,0,0,0,PRB001,PRB001(9)
23830,2021-07-17 19:59:00,20.784055,0.019544,Sitting/Standing,0,25,0,0,PRB001,PRB001(9)
23831,2021-07-17 20:00:00,21.366070,0.016361,Sitting/Standing,0,0,0,0,PRB001,PRB001(9)


In [93]:
NEW_ACTIVITY_DESCRIPTIONS = {
    0: "Ascending stairs",
    1: 'Cycling',
    2: 'Descending stairs',
    3: 'Lying on back',
    4: 'Lying on left side',
    5: 'Lying on right side',
    6: 'Lying on stomach',
    7: 'Movement',
    8: 'Not worn',
    9: 'Running',
    10: 'Shuffling',
    11: 'Sitting/Standing',
    12: 'Walking'
}
NEW_ACTIVITY_DESCRIPTIONS_INVERT = {v: k for k, v in NEW_ACTIVITY_DESCRIPTIONS.items()}

still_activity = ['Lying on back', 
                  'Lying on left side', 
                  'Lying on right side', 
                  'Lying on stomach',
                  'Not worn',
                  'Sitting/Standing']

def process_data(id):
    df = pd.read_csv(f"{data_path}/{id}_aggregated.csv", parse_dates=["timestamp"], index_col=["timestamp"])
    df_agg = pd.DataFrame()
    for visit in df["long_id"].unique():
        df_agg = df_agg.append(df[df["long_id"] == visit])
        df_agg.loc[len(df_agg)] = {}
    df_agg = df_agg[["breathingRate", "activityLevel", "activity_name"]]
    df_agg["activity_name_num"] = df_agg["activity_name"].map(NEW_ACTIVITY_DESCRIPTIONS_INVERT)
    df_agg["activity_binary"] = df_agg["activity_name"].apply(lambda x: np.nan if pd.isna(x) else (0 if x in still_activity else 1))
    df_agg["is_walking"] = df_agg["activity_name"].apply(lambda x: np.nan if pd.isna(x) else (1 if x == 'Walking' else 0))
    return df_agg

def process_data_small(id):
    df = pd.read_csv(f"{data_path}/{id}_aggregated.csv", parse_dates=["timestamp"], index_col=["timestamp"])
    ids = df["long_id"].unique()
    dfs = [pd.DataFrame]*len(ids)
    for i, visit in enumerate(ids):
        dfs[i] = df[df["long_id"] == visit]
        dfs[i] = dfs[i][["breathingRate", "activityLevel", "activity_name"]]
        dfs[i]["activity_name_num"] = dfs[i]["activity_name"].map(NEW_ACTIVITY_DESCRIPTIONS_INVERT)
        dfs[i]["activity_binary"] = dfs[i]["activity_name"].apply(lambda x: np.nan if pd.isna(x) else (0 if x in still_activity else 1))
        dfs[i]["is_walking"] = dfs[i]["activity_name"].apply(lambda x: np.nan if pd.isna(x) else (1 if x == 'Walking' else 0))
    return ids, dfs

process_data("PRB001")

,breathingRate,activityLevel,activity_name,activity_name_num,activity_binary,is_walking
timestamp,,,,,,
2021-07-07 14:56:00,NaN,0.124682,Movement,7.0,1.0,0.0
2021-07-07 14:57:00,16.119420,0.029450,Movement,7.0,1.0,0.0
2021-07-07 14:58:00,20.930561,0.120812,Movement,7.0,1.0,0.0
2021-07-07 14:59:00,18.133931,0.051179,Movement,7.0,1.0,0.0
2021-07-07 15:00:00,22.970766,0.105806,Lying on right side,5.0,0.0,0.0
...,...,...,...,...,...,...
2021-07-17 19:58:00,23.736564,0.022812,Sitting/Standing,11.0,0.0,0.0
2021-07-17 19:59:00,20.784055,0.019544,Sitting/Standing,11.0,0.0,0.0
2021-07-17 20:00:00,21.366070,0.016361,Sitting/Standing,11.0,0.0,0.0


In [98]:
datetimes = [pd.to_datetime('2023-10-30 10:00:00'),
             pd.to_datetime('2023-10-30 10:05:00'),
             pd.to_datetime('2023-10-30 10:10:00')]

# Calculate the average datetime
average_timestamp = sum(dt.timestamp() for dt in datetimes) / len(datetimes)
average_datetime = pd.to_datetime(average_timestamp, unit='s')

# Print the average datetime
print("Average Datetime:", average_datetime)

Average Datetime: 2023-10-30 10:05:00


In [101]:
ids, dfs = process_data_small("PRB001")
print(len(ids))
datetimes =[]
for id, df in zip(ids, dfs):
    datetimes.append(df.index.max() - df.index.min())
    print(id, df.index.max() - df.index.min())
pd.DataFrame(datetimes).mean()

59
PRB001(1) 0 days 06:41:00
PRB001(10) 0 days 11:18:00
PRB001(11) 0 days 13:21:00
PRB001(12) 0 days 12:04:00
PRB001(13) 0 days 10:08:00
PRB001(14) 0 days 13:09:00
PRB001(15) 0 days 18:31:00
PRB001(16) 0 days 21:51:00
PRB001(17) 0 days 14:25:00
PRB001(18) 0 days 21:03:00
PRB001(19) 0 days 14:14:00
PRB001(2) 0 days 09:54:00
PRB001(20) 0 days 14:08:00
PRB001(21) 0 days 16:31:00
PRB001(22) 0 days 16:00:00
PRB001(23) 1 days 00:00:00
PRB001(24) 0 days 22:01:00
PRB001(25) 0 days 14:35:00
PRB001(26) 0 days 20:20:00
PRB001(27) 0 days 20:07:00
PRB001(28) 0 days 21:37:00
PRB001(29) 1 days 00:00:00
PRB001(3) 0 days 02:08:00
PRB001(30) 1 days 00:00:00
PRB001(31) 1 days 00:00:00
PRB001(32) 0 days 21:56:00
PRB001(33) 0 days 13:00:00
PRB001(34) 0 days 18:14:00
PRB001(35) 0 days 13:57:00
PRB001(36) 0 days 14:58:00
PRB001(37) 1 days 00:00:00
PRB001(38) 0 days 20:33:00
PRB001(39) 0 days 12:44:00
PRB001(4) 0 days 11:30:00
PRB001(40) 0 days 11:26:00
PRB001(41) 0 days 13:02:00
PRB001(42) 0 days 20:22:00
PR

0   0 days 13:52:47.796610169
dtype: timedelta64[ns]

In [90]:
def link_assumptions(tau_max):
    links = {}
    # Breathing rate may be caused by activity,
    # Breathing rate must have a contemporaneous link with activity_level, as the effect.
    links[0] = {(i, -tau): "-?>" for i in [0, 1] for tau in range(1, tau_max+1)}
    links[0][(1,  0)] = "-->"
    links[1] = {(0, 0): "<--"}
    return links

def pcmci_plus(id, df, resolution, test, tau_max, pc_alpha=0.05, window_size=4):
    print(f"+++ Running {id} on {test} +++")
    output_folder = f"{output_path}/r{resolution}t{tau_max}/{test}/{id[:6]}"
        
#     if pathlib.Path(f"{output_folder}/{id}.pickle").exists():
#         print("  Already exist")
#         return
    
    # Read Processed data
    if len(df) - (df.isna().sum(axis=1) == 5).sum() < 10:
        print(f"No enough data for {id}")
        return
    
    # Handle missing values
    df.fillna(999., inplace=True)
    
    # Initialise whichever conditional indpendence test we are going to use.
    if test == "wls_parcorr":
        output_folder = output_folder + f"/w{window_size}"
        ci_test = ParCorrWLS(window_size=window_size)
        dataframe = pp.DataFrame(df.values, 
                                 var_names=df.columns.tolist(), 
                                 missing_flag=999.)
    elif test == "regression_CI":
        ci_test = RegressionCI(significance='analytic')
        type_mask = np.zeros(df.shape, dtype='int')
        # breathing rate is continuous, encoded as 0 in type_mask
        type_mask[:, 0] = 0
        # activity types are discrete, encoded as 1 in type_mask
        type_mask[:, 1] = 1
        dataframe = pp.DataFrame(df.values, 
                                 type_mask=type_mask, 
                                 var_names=df.columns.tolist(), 
                                 missing_flag=999.)
        
    # Run pcmci+
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=ci_test, verbosity=0)
#     results = pcmci.run_pcmciplus(tau_max=tau_max, pc_alpha=pc_alpha, link_assumptions=link_assumptions(tau_max))
    try:
        results = pcmci.run_pcmciplus(tau_max=tau_max, pc_alpha=pc_alpha, link_assumptions=link_assumptions(tau_max), reset_lagged_links=True)
        # reset_lagged_links=True
    except:
        print(f"  Not able to run pcmci+ for {id} on {test}")
        return
    
#     tp.plot_graph(graph=results['graph'], var_names=var_names)
#     plt.show()

    # Save result obejct
    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    outfile = open(f"{output_folder}/{id}.pickle", "wb")
    pickle.dump(results, outfile)
    outfile.close()
    print("  Done")

In [34]:
test = "wls_parcorr"; var_names = ["breathingRate", "activityLevel"]
resolution = 1; window_size = 15; tau_max = 30; pc_alpha = 0.05
# ids = [f[:6] for f in os.listdir(data_path)]
ids = ["PRB001", "PRB103", "PRB201"]

# Run the PCMCI+ algorithm
for id in ids:
    df = process_data(id)
    pcmci_plus(id, df[var_names], resolution, test, tau_max, pc_alpha, window_size)
#     clear_output()

+++ Running PRB103 on wls_parcorr +++
  Done
+++ Running PRB201 on wls_parcorr +++
  Done


In [59]:
pd.options.mode.chained_assignment = None

In [102]:
test = "wls_parcorr"; var_names = ["breathingRate", "activityLevel"]
resolution = 1; tau_max = 30; pc_alpha = 0.05; window_size = 4
ids, dfs = process_data_small("PRB001")

# Run the PCMCI+ algorithm
for id, df in zip(ids, dfs):
    pcmci_plus(id, df[var_names], resolution, test, tau_max, pc_alpha, window_size)
#     clear_output()

+++ Running PRB001(1) on wls_parcorr +++
  Done
+++ Running PRB001(10) on wls_parcorr +++
  Not able to run pcmci+ for PRB001(10) on wls_parcorr
+++ Running PRB001(11) on wls_parcorr +++
  Done
+++ Running PRB001(12) on wls_parcorr +++
  Done
+++ Running PRB001(13) on wls_parcorr +++
  Done
+++ Running PRB001(14) on wls_parcorr +++
  Done
+++ Running PRB001(15) on wls_parcorr +++
  Done
+++ Running PRB001(16) on wls_parcorr +++
  Done
+++ Running PRB001(17) on wls_parcorr +++
  Done
+++ Running PRB001(18) on wls_parcorr +++
  Done
+++ Running PRB001(19) on wls_parcorr +++
  Done
+++ Running PRB001(2) on wls_parcorr +++
  Done
+++ Running PRB001(20) on wls_parcorr +++
  Done
+++ Running PRB001(21) on wls_parcorr +++
  Done
+++ Running PRB001(22) on wls_parcorr +++
  Done
+++ Running PRB001(23) on wls_parcorr +++
  Done
+++ Running PRB001(24) on wls_parcorr +++
  Done
+++ Running PRB001(25) on wls_parcorr +++
  Done
+++ Running PRB001(26) on wls_parcorr +++
  Done
+++ Running PRB001(27) 

In [91]:
test = "regression_CI"; var_names = ["breathingRate", "activity_binary"]
resolution = 1; tau_max = 30; pc_alpha = 0.05
ids, dfs = process_data_small("PRB001")

# Run the PCMCI+ algorithm
for id, df in zip(ids, dfs):
    pcmci_plus(id, df[var_names], resolution, test, tau_max, pc_alpha, window_size)
#     clear_output()

+++ Running PRB001(1) on regression_CI +++
  Done
+++ Running PRB001(10) on regression_CI +++
  Not able to run pcmci+ for PRB001(10) on regression_CI
+++ Running PRB001(11) on regression_CI +++
  Done
+++ Running PRB001(12) on regression_CI +++
  Done
+++ Running PRB001(13) on regression_CI +++
  Done
+++ Running PRB001(14) on regression_CI +++
  Done
+++ Running PRB001(15) on regression_CI +++
  Done
+++ Running PRB001(16) on regression_CI +++
  Done
+++ Running PRB001(17) on regression_CI +++
  Done
+++ Running PRB001(18) on regression_CI +++
  Done
+++ Running PRB001(19) on regression_CI +++
  Done
+++ Running PRB001(2) on regression_CI +++
  Done
+++ Running PRB001(20) on regression_CI +++
  Done
+++ Running PRB001(21) on regression_CI +++
  Done
+++ Running PRB001(22) on regression_CI +++
  Done
+++ Running PRB001(23) on regression_CI +++
  Done
+++ Running PRB001(24) on regression_CI +++
  Done
+++ Running PRB001(25) on regression_CI +++
  Done
+++ Running PRB001(26) on regressi

d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistes

  Done
+++ Running PRB001(39) on regression_CI +++
  Done
+++ Running PRB001(4) on regression_CI +++


d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosiste

d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosiste

...

d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosiste

  Done
+++ Running PRB001(58) on regression_CI +++


d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosiste

d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
d:\programs\python\python3.8.10\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest o

  Done
+++ Running PRB001(59) on regression_CI +++
  Not able to run pcmci+ for PRB001(59) on regression_CI
+++ Running PRB001(6) on regression_CI +++
  Not able to run pcmci+ for PRB001(6) on regression_CI
+++ Running PRB001(60) on regression_CI +++
  Not able to run pcmci+ for PRB001(60) on regression_CI
+++ Running PRB001(7) on regression_CI +++
  Not able to run pcmci+ for PRB001(7) on regression_CI
+++ Running PRB001(9) on regression_CI +++
  Done
